In [1]:
## started this...

include("MDPModelFunctions2.jl")
using PyCall

pb = pyimport("pybads")
pyimport("sys").stdout.flush()
#using BlackBoxOptim

In [5]:
using BayesianOptimization, GaussianProcesses, Distributions

model = ElasticGPE(2,                            # 2 input dimensions
                   mean = MeanConst(0.),         
                   kernel = SEArd([0., 0.], 5.),
                   logNoise = 0.,
                   capacity = 3000)              # the initial capacity of the GP is 3000 samples.
set_priors!(model.mean, [Normal(1, 2)])

1-element Vector{Normal{Float64}}:
 Normal{Float64}(μ=1.0, σ=2.0)

In [1]:
using Pkg
Pkg.add("BayesianOptimization")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed JpegTurbo_jll ────────── v3.0.1+0
   Installed ProgressMeter ────────── v1.9.0
   Installed Unitful ──────────────── v1.19.0
   Installed ScikitLearnBase ──────── v0.5.0
   Installed PDMats ───────────────── v0.10.1
   Installed StaticArrays ─────────── v1.9.1
   Installed FastGaussQuadrature ──── v0.4.9
   Installed MutableArithmetics ───── v1.4.0
   Installed PlotUtils ────────────── v1.4.0
   Installed Distances ────────────── v0.10.11
   Installed HTTP ─────────────────── v1.10.1
   Installed FiniteDiff ───────────── v2.22.0
   Installed JLD2 ─────────────────── v0.4.45
   Installed FileIO ───────────────── v1.16.2
   Installed ElasticPDMats ────────── v0.2.2
   Installed BenchmarkTools ───────── v1.4.0
   Installed MathOptInterface ─────── v1.2.0
   Installed Bzip2_jll ────────────── v1.0.8+1
   Installed fzf_jll ──────────────── v0.43.0+0
   Installed GLFW_jll ───────────────

In [4]:
Pkg.add("GaussianProcesses")

   Resolving package versions...
    Updating `~/Dropbox/Griffiths_Lab_Stuff/Code/Memory_Models/Project.toml`
  [891a1506] + GaussianProcesses v0.12.5
  No Changes to `~/Dropbox/Griffiths_Lab_Stuff/Code/Memory_Models/Manifest.toml`


In [2]:
#using Pkg
#Pkg.add("BlackBoxOptim")

In [3]:
Threads.nthreads()

1

In [11]:

function format_res_cowan_k(res; exp_num = 1)
    
    if exp_num == 1
    
        # row is 2 obj vs 4 obj
        # col is pre-cue, neutral, retro-cue, neutral
        
        part_pcorr = res;
        model_cowan_p_corr = [part_pcorr; part_pcorr[2,:]']'
        model_cowan_k_mtx = cowan_k(model_cowan_p_corr, 2)
        model_cowan_k_mtx[2,:] = cowan_k(model_cowan_p_corr[2,:], 4)
        
        model_cowan_k = [model_cowan_k_mtx[1,:]' model_cowan_k_mtx[2,:]']';#[model_cowan_k_mtx[1,:] model_cowan_k_mtx[2,:]]
    elseif exp_num == 2
        
        # IM cued, IM neutral, VSTM cued, VSTM neutral - need to adjust from what it was saved as
        
        (p_short_neutral, p_short_retro, p_long_neutral, p_long_retro) = res
        model_cowan_k = cowan_k([p_short_retro p_short_neutral p_long_retro p_long_neutral],4)'
        
    else
        
        (p_IM_neutral, p_IM_retro, p_VSTM_neutral, p_VSTM_retro, p_Long_VSTM_neutral, p_Long_VSTM_retro) = res

        # Load 3 cued, Load 3 Neutral, Load 6 cued, Load 6 neutral
        IM_model = [cowan_k(p_IM_retro[1],3) cowan_k(p_IM_neutral[1],3) cowan_k(p_IM_retro[2],6) cowan_k(p_IM_neutral[2], 6)]
        Short_VSTM_model = [cowan_k(p_VSTM_retro[1],3) cowan_k(p_VSTM_neutral[1],3) cowan_k(p_VSTM_retro[2],6) cowan_k(p_VSTM_neutral[2], 6)]
        Long_VSTM_model = [cowan_k(p_Long_VSTM_retro[1],3) cowan_k(p_Long_VSTM_neutral[1],3) cowan_k(p_Long_VSTM_retro[2],6) cowan_k(p_Long_VSTM_neutral[2], 6)]

        # 3 x 4
        model_cowan_k = [IM_model; Short_VSTM_model; Long_VSTM_model]  
        
    end
        
    return model_cowan_k
    
end    


# 6 parameteres for 44 datapoints 

exp1_cowan_k_true_7_yr = [1.16 1.14 1.13 1.02 2.80 .91 1.30 .96]' # 8
exp1_cowan_k_true_adult = [1.98 1.94 1.97 1.95 3.87 2.66 3.64 2.87]'

# experiment 2 - 4 objects
# col is IM cued, IM neutral, VSTM cued, VSTM neutral

exp2_cowan_k_true_7_yr = [1.55 1.16 1.00 0.69]' # 4
exp2_cowan_k_true_adult = [3.54 2.58 3.58 2.65]'

# experiment 3

# Load 3 cued, Load 3 Neutral, Load 6 cued, Load 6 neutral
IM_true_7_yr = [1.44 1.27 1.82 0.88]
Short_VSTM_true_7_yr = [1.14 0.77 NaN NaN]
Long_VSTM_true_7_yr = [1.19 0.90 NaN NaN]

# 8

exp3_cowan_k_true_7_yr = [IM_true_7_yr; Short_VSTM_true_7_yr; Short_VSTM_true_7_yr]


IM_true_adult = [2.63 2.41 4.80 2.46]
Short_VSTM_true_adult = [2.75 2.38 3.97 2.15]
Long_VSTM_true_adult = [2.76 2.30 3.65 1.75]
# 12

exp3_cowan_k_true_adult = [IM_true_adult; Short_VSTM_true_adult; Short_VSTM_true_adult]

3×4 Matrix{Float64}:
 2.63  2.41  4.8   2.46
 2.75  2.38  3.97  2.15
 2.75  2.38  3.97  2.15

In [5]:
# function to compute mean squared error...

# takes in Adult_Quanta (int), Adult_Epsilon (0-1 bounded), Child_Quanta (int), Child_Epsilon (0-1 bounded), Mem_slope (less than 1), NT_val # 6 params...

# params = [20, .8, 5, .4, .1, 100]

function compute_mse(params, exp1_cowan_k_true_7_yr, exp1_cowan_k_true_adult, exp2_cowan_k_true_7_yr, exp2_cowan_k_true_adult, exp3_cowan_k_true_7_yr, exp3_cowan_k_true_adult)

    adult_quanta = round(100*params[1])
    adult_epsilon = params[2]
    child_quanta = round(100*params[3])
    child_epsilon = params[4]
    mem_slope = params[5]
    nt_val = Int(round(1000*params[6]))

    N_Trials = 200

    # simulate for children...
    children_res_1 = sim_exp1(child_epsilon, child_quanta, nt_val; mem_slope = mem_slope, N_Trials = N_Trials);
    children_cowan_k1 = format_res_cowan_k(children_res_1; exp_num = 1)
    children_res_2 = sim_exp2(child_epsilon, child_quanta, nt_val; mem_slope = mem_slope, N_Trials = N_Trials);
    children_cowan_k2 = format_res_cowan_k(children_res_2; exp_num = 2)
    children_res_3 = sim_exp3(child_epsilon, child_quanta, nt_val; mem_slope = mem_slope, N_Trials = N_Trials);
    children_cowan_k3 = format_res_cowan_k(children_res_3; exp_num = 3)


    adult_res_1 = sim_exp1(adult_epsilon, adult_quanta, nt_val; mem_slope = mem_slope, N_Trials = N_Trials);
    adult_cowan_k1 = format_res_cowan_k(adult_res_1; exp_num = 1)
    adult_res_2 = sim_exp2(adult_epsilon, adult_quanta, nt_val; mem_slope = mem_slope, N_Trials = N_Trials);
    adult_cowan_k2 = format_res_cowan_k(adult_res_2; exp_num = 2)
    adult_res_3 = sim_exp3(adult_epsilon, adult_quanta, nt_val; mem_slope = mem_slope, N_Trials = N_Trials);
    adult_cowan_k3 = format_res_cowan_k(adult_res_3; exp_num = 3)

    child_mse1 = sum((children_cowan_k1 .- exp1_cowan_k_true_7_yr).^2)
    adult_mse1 = sum((adult_cowan_k1 .- exp1_cowan_k_true_adult).^2)

    child_mse2 = sum((children_cowan_k2 .- exp2_cowan_k_true_7_yr).^2)
    adult_mse2 = sum((adult_cowan_k2 .- exp2_cowan_k_true_adult).^2)

    child_sq_err = (children_cowan_k3[:] .- exp3_cowan_k_true_7_yr[:]).^2
    child_mse3 = sum(child_sq_err[.!isnan.(child_sq_err)])
    adult_mse3 = sum((adult_cowan_k3 .- exp3_cowan_k_true_adult).^2)

    child_mse = child_mse1 + child_mse2 + child_mse3
    adult_mse = adult_mse1 + adult_mse2 + adult_mse3

    both_mse = child_mse + adult_mse
    
    return both_mse
    
end

compute_mse (generic function with 1 method)

In [6]:
params = [.2, .8, .5, .4, .1, .1]
compute_mse(params, exp1_cowan_k_true_7_yr, exp1_cowan_k_true_adult, exp2_cowan_k_true_7_yr, exp2_cowan_k_true_adult, exp3_cowan_k_true_7_yr, exp3_cowan_k_true_adult)

96.44160509069798

In [ ]:
target(params) = compute_mse(params, exp1_cowan_k_true_7_yr, exp1_cowan_k_true_adult, exp2_cowan_k_true_7_yr, exp2_cowan_k_true_adult, exp3_cowan_k_true_7_yr, exp3_cowan_k_true_adult)
bboptimize(target; NumDimensions = 6, SearchRange = (0.0,1.0), MaxTime=300.0)

Starting optimization with optimizer DiffEvoOpt{FitPopulation{Float64}, RadiusLimitedSelector, BlackBoxOptim.AdaptiveDiffEvoRandBin{3}, RandomBound{ContinuousRectSearchSpace}}
0.00 secs, 0 evals, 0 steps
71.00 secs, 2 evals, 1 steps, improv/step: 1.000 (last = 1.0000), fitness=15.427245176
91.06 secs, 4 evals, 2 steps, improv/step: 0.500 (last = 0.0000), fitness=15.427245176
138.01 secs, 6 evals, 3 steps, improv/step: 0.667 (last = 1.0000), fitness=15.427245176
185.25 secs, 8 evals, 4 steps, improv/step: 0.500 (last = 0.0000), fitness=15.427245176
187.58 secs, 10 evals, 5 steps, improv/step: 0.400 (last = 0.0000), fitness=15.427245176
247.78 secs, 12 evals, 6 steps, improv/step: 0.333 (last = 0.0000), fitness=15.427245176


┌ Warning: Optimization interrupted, recovering intermediate results...
└ @ BlackBoxOptim ~/.julia/packages/BlackBoxOptim/lZtsr/src/opt_controller.jl:483


In [21]:
options = Dict("uncertainty_handling" => true, "max_fun_evals" => 40,"noise_final_samples" => 2 )

Dict{String, Integer} with 3 entries:
  "noise_final_samples"  => 2
  "uncertainty_handling" => true
  "max_fun_evals"        => 40

In [22]:
# takes in Adult_Quanta (int), Adult_Epsilon (0-1 bounded), Child_Quanta (int), Child_Epsilon (0-1 bounded), Mem_slope (less than 1), NT_val # 6 params...
lb = [2, 0, 2, 0, .001, 10]
ub = [100, 1, 100, 1, 1, 1000]
plausible_lower_bounds = [10, .8, 2, .8, .05, 500]
plausible_upper_bounds = [100, 1, 100, 1, .6, 1000]

target(params) = compute_mse(params, exp1_cowan_k_true_7_yr, exp1_cowan_k_true_adult, exp2_cowan_k_true_7_yr, exp2_cowan_k_true_adult, exp3_cowan_k_true_7_yr, exp3_cowan_k_true_adult)
x0 = [20,.9, 20, .9, .1, 600]

bads = pb.BADS(target, x0, lb, ub, plausible_lower_bounds, plausible_upper_bounds, options = options)

bads:TooCloseBounds: For each variable, hard and plausible bounds should not be too close. Moving plausible bounds.
Variables (index) internally transformed to log coordinates: [[0 2]
 [0 4]]


PyObject <pybads.bads.bads.BADS object at 0x293a56380>

In [23]:
bads.optimize()


Beginning optimization of a STOCHASTIC objective function

 Iteration    f-count      E[f(x)]        SD[f(x)]           MeshScale          Method              Actions
     0           1         194.445             nan               1                                  
     0          33         52.9346             nan               1          Initial mesh            Initial points
     0          38         28.0739         1.01263               1        Successful poll           Train
Optimization terminated: reached maximum number of function evaluations options['max_fun_evals'].
Observed function value at minimum: 28.040573702451134 (1 sample). Estimated: 28.07390370170654 ± 1.0126302652810046 (GP mean ± SEM).


LoadError: PyError ($(Expr(:escape, :(ccall(#= /Users/erussek/.julia/packages/PyCall/1gn3u/src/pyfncall.jl:43 =# @pysym(:PyObject_Call), PyPtr, (PyPtr, PyPtr, PyPtr), o, pyargsptr, kw))))) <class 'TypeError'>
TypeError("cannot pickle 'PyCall.jlwrap' object")
  File "/Users/erussek/.julia/conda/3/lib/python3.10/site-packages/pybads/bads/bads.py", line 1518, in optimize
    optimize_result = OptimizeResult(self)
  File "/Users/erussek/.julia/conda/3/lib/python3.10/site-packages/pybads/bads/optimize_result.py", line 81, in __init__
    self.set_attributes(bads)
  File "/Users/erussek/.julia/conda/3/lib/python3.10/site-packages/pybads/bads/optimize_result.py", line 90, in set_attributes
    self["fun"] = bads.function_logger.fun
  File "/Users/erussek/.julia/conda/3/lib/python3.10/site-packages/pybads/bads/optimize_result.py", line 177, in __setitem__
    dict.__setitem__(self, key, copy.deepcopy(val))
  File "/Users/erussek/.julia/conda/3/lib/python3.10/copy.py", line 161, in deepcopy
   

In [15]:
optimize_result

LoadError: UndefVarError: `optimize_result` not defined

In [12]:
function compute_mse(params, exp1_cowan_k_true_7_yr, exp1_cowan_k_true_adult, exp2_cowan_k_true_7_yr, exp2_cowan_k_true_adult, exp3_cowan_k_true_7_yr, exp3_cowan_k_true_adult)

    adult_quanta = round(100*params[1])
    adult_epsilon = params[2]
    mem_slope = .2
    nt_val = 1600
    
    #child_quanta = round(100*params[3])
    #child_epsilon = params[4]
    #mem_slope = params[5]
    #nt_val = Int(round(1000*params[6]))

    N_Trials = 500
    
    

    """
    
    # simulate for children...
    children_res_1 = sim_exp1(child_epsilon, child_quanta, nt_val; mem_slope = mem_slope, N_Trials = N_Trials);
    children_cowan_k1 = format_res_cowan_k(children_res_1; exp_num = 1)
    children_res_2 = sim_exp2(child_epsilon, child_quanta, nt_val; mem_slope = mem_slope, N_Trials = N_Trials);
    children_cowan_k2 = format_res_cowan_k(children_res_2; exp_num = 2)
    children_res_3 = sim_exp3(child_epsilon, child_quanta, nt_val; mem_slope = mem_slope, N_Trials = N_Trials);
    children_cowan_k3 = format_res_cowan_k(children_res_3; exp_num = 3)
    
    """


    adult_res_1 = sim_exp1(adult_epsilon, adult_quanta, nt_val; mem_slope = mem_slope, N_Trials = N_Trials);
    adult_cowan_k1 = format_res_cowan_k(adult_res_1; exp_num = 1)
    adult_res_2 = sim_exp2(adult_epsilon, adult_quanta, nt_val; mem_slope = mem_slope, N_Trials = N_Trials);
    adult_cowan_k2 = format_res_cowan_k(adult_res_2; exp_num = 2)
    adult_res_3 = sim_exp3(adult_epsilon, adult_quanta, nt_val; mem_slope = mem_slope, N_Trials = N_Trials);
    adult_cowan_k3 = format_res_cowan_k(adult_res_3; exp_num = 3)

    #child_mse1 = sum((children_cowan_k1 .- exp1_cowan_k_true_7_yr).^2)
    adult_mse1 = sum((adult_cowan_k1 .- exp1_cowan_k_true_adult).^2)

    #child_mse2 = sum((children_cowan_k2 .- exp2_cowan_k_true_7_yr).^2)
    adult_mse2 = sum((adult_cowan_k2 .- exp2_cowan_k_true_adult).^2)

    #child_sq_err = (children_cowan_k3[:] .- exp3_cowan_k_true_7_yr[:]).^2
    #child_mse3 = sum(child_sq_err[.!isnan.(child_sq_err)])
    adult_mse3 = sum((adult_cowan_k3 .- exp3_cowan_k_true_adult).^2)

    #child_mse = child_mse1 + child_mse2 + child_mse3
    adult_mse = adult_mse1 + adult_mse2 + adult_mse3

    #both_mse = child_mse + adult_mse
    
    return adult_mse
    
end


compute_mse (generic function with 1 method)

In [13]:
target(params) = compute_mse(params, exp1_cowan_k_true_7_yr, exp1_cowan_k_true_adult, exp2_cowan_k_true_7_yr, exp2_cowan_k_true_adult, exp3_cowan_k_true_7_yr, exp3_cowan_k_true_adult)


target (generic function with 1 method)

In [14]:
target([.5,.5])

LoadError: InterruptException: